# 02 Global Network Summaries

In [1]:
# ── Path-Fix for Imports ───────────────────────────────────────────────────
import sys
from pathlib import Path
# Prepend parent directory (repo root) to sys.path
sys.path.insert(0, str(Path().resolve().parent))

# ── Standard Imports ───────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from scripts.graph_utils import load_clean_df, build_unweighted_graph, build_weighted_graph

sns.set(style="whitegrid")

## Analysis goes here…

In [2]:
THRESHOLD  = 0
START_DATE = "2023-04-14"
END_DATE   = "2023-05-31"
USE_CACHE  = True

In [3]:
# Load Graph (Unweighted)
df = load_clean_df(
    threshold=THRESHOLD,
    start_date=START_DATE,
    end_date=END_DATE
)
G = build_unweighted_graph(
    df,
    threshold=THRESHOLD,
    start_date=START_DATE,
    end_date=END_DATE,
    use_cache=USE_CACHE
)

print(f"Total nodes: {G.number_of_nodes():,}")
print(f"Total edges: {G.number_of_edges():,}")
print(f"Density: {nx.density(G):.2e}")

Total nodes: 215,741
Total edges: 426,954
Density: 9.17e-06


In [4]:
# Connected Components

# Weakly connected components
wcc = list(nx.weakly_connected_components(G))
wcc_sizes = np.array([len(c) for c in wcc])

# Strongly connected components
scc = list(nx.strongly_connected_components(G))
scc_sizes = np.array([len(c) for c in scc])

print(f"# Weakly CC: {len(wcc)}; Giant WCC size: {wcc_sizes.max():,}")
print(f"# Strongly CC: {len(scc)}; Giant SCC size: {scc_sizes.max():,}")

# Extract giant WCC and SCC
giant_wcc = max(wcc, key=len)
core_scc = max(scc, key=len)


# Weakly CC: 8; Giant WCC size: 215,727
# Strongly CC: 97852; Giant SCC size: 117,717


In [5]:
# Approximate Giant WCC Metrics

import random
from tqdm import tqdm
from networkx.algorithms.approximation import diameter as approx_diameter

# work on the undirected giant WCC
gG = G.subgraph(giant_wcc).copy().to_undirected()

# 1) Approximate average shortest path via sampling
sample_size = 500
seeds = random.sample(list(gG.nodes()), sample_size)
all_dists = []
for u in tqdm(seeds, desc="Sampling BFS"):
    all_dists.extend(nx.single_source_shortest_path_length(gG, u).values())
approx_avg = sum(all_dists) / len(all_dists)
print(f"Approx avg shortest path (n={sample_size} seeds): {approx_avg:.2f}")

# 2) Approximate diameter via double‐sweep
u0 = next(iter(gG.nodes()))
u1 = max(nx.single_source_shortest_path_length(gG, u0), key=lambda x: x[1])[0]
diam_sweep = max(nx.single_source_shortest_path_length(gG, u1).values())
print(f"Approx diameter (double‐sweep): {diam_sweep}")

# 3) Optional bounds from NetworkX
low, high = approx_diameter(gG, use_bounds=True)
print(f"Diameter bounds: [{low}, {high}]")


KeyboardInterrupt: 

In [ ]:
# Component Size Distributions

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Weakly CC sizes
ax[0].hist(wcc_sizes, bins=50, log=True)
ax[0].set_xscale('log'); ax[0].set_title('Weakly CC Sizes (log)')

# Strongly CC sizes
ax[1].hist(scc_sizes, bins=50, log=True)
ax[1].set_xscale('log'); ax[1].set_title('Strongly CC Sizes (log)')
plt.tight_layout()
plt.show()

In [ ]:
# Bonus Bow-Tie Decomposition

# Build reverse graph
G_rev = G.reverse(copy=False)
# Compute reachable sets
T_out = set()
for n in core_scc:
    T_out |= nx.descendants(G, n)
T_in = set()
for n in core_scc:
    T_in |= nx.descendants(G_rev, n)
# Define regions
IN       = T_in - core_scc
OUT      = T_out - core_scc
TUBES    = T_in & T_out
TENDRILS = set(giant_wcc) - core_scc - IN - OUT - TUBES

# Summary table
bowtie = {
    'IN':       len(IN),
    'SCC':      len(core_scc),
    'OUT':      len(OUT),
    'TUBES':    len(TUBES),
    'TENDRILS': len(TENDRILS)
}
bowtie_df = pd.DataFrame.from_dict(bowtie, orient='index', columns=['size'])
display(bowtie_df)